In [2]:
from transformers import AutoTokenizer,AutoModelForQuestionAnswering
from datasets import load_from_disk

tok=AutoTokenizer.from_pretrained("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\distill_bert_uncased_Mask_QnA")

model=AutoModelForQuestionAnswering.from_pretrained("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\distill_bert_uncased_Mask_QnA")

squad=load_from_disk("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\SQUAD dataset")

Some weights of the model checkpoint at D:\IVY Batches\MY ML DL Projects\Transformers Models\distill_bert_uncased_Mask_QnA were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at D:\IVY Batches\MY ML DL Projects\Transformers Models\distill_bert_un

In [5]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tok(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [6]:
preprocess_tokens=squad.map(preprocess_function,batched=True,remove_columns=squad['train'].column_names)

100%|██████████| 11/11 [00:10<00:00,  1.06ba/s]


In [7]:
preprocess_tokens.save_to_disk("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\Processed Squad")

In [12]:
from transformers import DefaultDataCollator,Trainer

collator=DefaultDataCollator()

train=Trainer(model=model,train_dataset=preprocess_tokens['train'],
            eval_dataset=preprocess_tokens['validation'],tokenizer=tok,data_collator=collator)

### Context Generation

In [3]:
from transformers import GPT2Tokenizer,TFAutoModelForCausalLM

tokenize=GPT2Tokenizer.from_pretrained("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\GPT2 Text gen")

model=TFAutoModelForCausalLM.from_pretrained("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\GPT2 Text gen")

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at D:\IVY Batches\MY ML DL Projects\Transformers Models\GPT2 Text gen.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
from transformers import pipeline

pipe=pipeline(task='text-generation',model=model,tokenizer=tokenize)

In [15]:
text=pipe("I wanted to",max_length=100,do_samples=False)

text

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[{'generated_text': 'I wanted to create something like this because I feel it\'s cool. But I thought in the beginning that it was a great idea.\n\n\nWhen I got an idea for the project, I met with many other people and they said, \'How can I get this idea without this?\'" We were actually all very interested in this idea because that was the first time I did things with The Lord of the Rings. We did things with the Lord of the Rings, but had nothing to do with'}]

In [19]:
print(text[0]['generated_text'].strip())

I wanted to create something like this because I feel it's cool. But I thought in the beginning that it was a great idea.


When I got an idea for the project, I met with many other people and they said, 'How can I get this idea without this?'" We were actually all very interested in this idea because that was the first time I did things with The Lord of the Rings. We did things with the Lord of the Rings, but had nothing to do with


#### QnA

In [6]:
from transformers import AutoTokenizer,TFAutoModelForQuestionAnswering

token=AutoTokenizer.from_pretrained("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\Full trained squad QA")

qa_model=TFAutoModelForQuestionAnswering.from_pretrained("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\Full trained squad QA")

Some layers from the model checkpoint at D:\IVY Batches\MY ML DL Projects\Transformers Models\Full trained squad QA were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at D:\IVY Batches\MY ML DL Projects\Transformers Models\Full trained squad QA and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [8]:
from transformers import pipeline
pipe=pipeline("question-answering",model=qa_model,tokenizer=token)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [10]:
pipe({'question':'what is capital of america?','context':'washington dc is capital of america'})

{'score': 0.9430935978889465, 'start': 0, 'end': 13, 'answer': 'washington dc'}

In [128]:
from datasets import load_from_disk

data=load_from_disk("D:\\IVY Batches\\MY ML DL Projects\\Transformers Models\\SQUAD dataset")

In [132]:
data['validation'][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


### Auto Suggest

In [1]:
from transformers import GPT2Tokenizer,TFAutoModelForCausalLM,pipelines

tokenize=GPT2Tokenizer.from_pretrained("D:\\IVY Batches\MY ML DL Projects\\Transformers Models\\GPT2 Text gen")
sug_model=TFAutoModelForCausalLM.from_pretrained("D:\\IVY Batches\MY ML DL Projects\\Transformers Models\\GPT2 Text gen")


<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at D:\IVY Batches\MY ML D

In [117]:
from transformers import tf_top_k_top_p_filtering
import tensorflow as tf

text=f"Are you going to get"

tokens=tokenize(text,return_tensors='tf')

inp_id=tokens['input_ids']

next_logit=sug_model(**tokens).logits[:,-1,:]

filter_next_logit=tf_top_k_top_p_filtering(next_logit,top_k=100,top_p=1.0)

random_choice=tf.random.categorical(filter_next_logit,dtype=tf.int32,num_samples=5)

generate=tf.concat([inp_id,random_choice],axis=1)

tokenize.decode(generate.numpy().tolist()[0]).split()[-5:]


['a', 'on', 'a', 'into', 'something']